In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm_notebook
import cv2

import keras
from keras.layers.convolutional import Conv2DTranspose
from keras.layers.merge import concatenate
from keras.layers import UpSampling2D, Conv2D, Activation, Input, Dropout, MaxPooling2D, Input
from keras import Model
from keras import backend as K
from keras.layers.core import Lambda
from keras.applications.resnet50 import ResNet50

#change path to train and test images
train_path = './train_images/'
test_path = './test_images/'


tr = pd.read_csv('train.csv')
df_train = tr[tr['EncodedPixels'].notnull()].reset_index(drop=True)

Simple RLE decoder

In [7]:
def rle2mask(rle, imgshape):
    width = imgshape[0]
    height= imgshape[1]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return np.flipud( np.rot90( mask.reshape(height, width), k=1 ) )

In [10]:
img_size = 256
def keras_generator(batch_size):
    while True:
        x_batch = []
        y_batch = []
        
        for i in range(batch_size):            
            fn = df_train['ImageId_ClassId'].iloc[i].split('_')[0]
            img = cv2.imread(train_path+fn)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)            
            
            mask = rle2mask(df_train['EncodedPixels'].iloc[i], img.shape)
            
            img = cv2.resize(img, (img_size, img_size))
            mask = cv2.resize(mask, (img_size, img_size))
            
            x_batch += [img]
            y_batch += [mask]
                                    
        x_batch = np.array(x_batch)
        y_batch = np.array(y_batch)

        yield x_batch, np.expand_dims(y_batch, -1)

Initialize ResNet model with ImageNet weights and compile it



In [21]:
weights = 'resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
img_shape = (256,256,3)
img_input = Input(shape=img_shape)
base_model = ResNet50(weights=weights, input_shape=img_shape, include_top=False,
                     input_tensor=img_input, classes=None)
base_out = base_model.output

up = Conv2D(1, (1,1), strides=(1, 1))(base_out)
up = UpSampling2D(size=(32, 32), interpolation='bilinear')(up)


model = Model(input=base_model.input, output=up)

adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(adam, 'binary_crossentropy')
#base_model.summary()

C:\Users\wanko\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("up...)`
  
W0908 00:10:45.255834 16644 deprecation_wrapper.py:119] From C:\Users\wanko\Anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0908 00:10:45.267805 16644 deprecation.py:323] From C:\Users\wanko\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train model and submit predictions 

In [32]:
batch_size = 16
model.fit_generator(keras_generator(batch_size),
              steps_per_epoch=100,
              epochs=20
              )

In [25]:
testfiles=os.listdir(test_path)

test_img = []
for fn in tqdm_notebook(testfiles):
        img = cv2.imread( test_path+fn )
        img = cv2.resize(img,(img_size,img_size))       
        test_img.append(img)
        

predict = model.predict(np.asarray(test_img))

In [26]:
def mask2rle(img):
    tmp = np.rot90( np.flipud( img ), k=3 )
    rle = []
    lastColor = 0;
    startpos = 0
    endpos = 0

    tmp = tmp.reshape(-1,1)   
    for i in range( len(tmp) ):
        if (lastColor==0) and tmp[i]>0:
            startpos = i
            lastColor = 1
        elif (lastColor==1)and(tmp[i]==0):
            endpos = i-1
            lastColor = 0
            rle.append( str(startpos)+' '+str(endpos-startpos+1) )
    return " ".join(rle)

In [27]:
pred_rle = []
for img in tqdm_notebook(predict):      
    img = cv2.resize(img, (1600, 256))
    tmp = np.copy(img)
    tmp[tmp<np.mean(img)] = 0
    tmp[tmp>0] = 1
    pred_rle.append(mask2rle(tmp))

In [28]:
sub = pd.read_csv( 'sample_submission.csv' )
for fn, rle in zip(testfiles, pred_rle):
    sub['EncodedPixels'][sub['ImageId_ClassId'].apply(lambda x: x.split('_')[0]) == fn] = rle
    
img_s = cv2.imread( test_path + sub['ImageId_ClassId'][16].split('_')[0])
mask_s = rle2mask(sub['EncodedPixels'][16], (256, 1600))

In [29]:
sub.to_csv('submission.csv', index=False)